## Setup

In [1]:
# Get raw advent-of-code data
from aocd.models import Puzzle

puzzle = Puzzle(year=2025, day=7)
input_data = puzzle.input_data
example = puzzle.examples[0]

In [2]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from common.utils.perf_check import time_solution

## Part a
Initially I thought I'd had to model the beam paths using DFS, but on further consideration, an iterative approach tracking the number of beams in each column should work better.

We can store information about each column's beam status as we go down the rows. Each time a beam hits a splitter, it splits into two beams going to the adjacent columns. We can keep track of the total number of splits and the number of beams reaching the bottom.


In [3]:
# Constants
SPLITTER = "^"

In [30]:
# Functions
def count_beam_splits(input_data: str) -> tuple[int, int]:
    """Count the total number of beam splits based on splitter locations."""
    lines = input_data.splitlines()
    field_width = len(lines[0])

    # Parse splitter locations as (y, x) tuples
    splitters = [(y, x) for y, line in enumerate(lines) for x, char in enumerate(line) if char == SPLITTER]

    # Beam presence per column; start with 1 at the first splitter's column
    col_path_count = [0] * field_width
    col_path_count[min(splitters)[1]] = 1

    # Initialize total split count and unique split locations
    total_split_count = 0

    # Process splitters from top to bottom
    for _, x in splitters:
        if col_path_count[x] > 0:
            # There is currently a beam in this column, so it will split
            total_split_count += 1

            # Create beams in adjacent columns, updating path counts
            col_path_count[x - 1] += col_path_count[x]
            col_path_count[x + 1] += col_path_count[x]

            # The splitter deactivates the beam in this column
            col_path_count[x] = 0

    return total_split_count, sum(col_path_count)

In [ ]:
# Correctness check
str(count_beam_splits(example.input_data)[0]) == example.answer_a

True

In [11]:
# Performance check
iterative_time_a = time_solution(count_beam_splits, input_data)
print(f"The iterative implementation takes {iterative_time_a:.2f} ms per run.")

The iterative implementation takes 0.54 ms per run.


In [ ]:
# Submit answer
puzzle.answer_a = count_beam_splits(input_data)

That's the right answer!  You are one gold star closer to decorating the North Pole. [Continue to Part Two]


## Part b
This seems like a DFS problem.


**Update**: On closer inspection, it's more of a dynamic programming problem, where we track the number of paths reaching each column. I've added a couple lines of code to part a's solution to implement this. This is also roughly 4 times faster than the DFS approach.

In [34]:
# Correctness check
str(count_beam_splits(example.input_data)[1]) == example.answer_b

True

In [35]:
# Performance check
iterative_time_a = time_solution(count_beam_splits, input_data)
print(f"The iterative implementation takes {iterative_time_a:.2f} ms per run.")

The iterative implementation takes 0.57 ms per run.


### Old DFS approach
If you're interested, here's the old DFS approach I considered before realizing the DP solution

In [12]:
# Functions
def count_tachyon_paths(input_data: str) -> int:
    """Count distinct timeline endpoints using BFS traversal."""
    lines = input_data.splitlines()
    field_width = len(lines[0])
    field_height = len(lines)

    # Parse splitter locations as a set of (y, x)
    splitters = {(y, x) for y, line in enumerate(lines) for x, ch in enumerate(line) if ch == SPLITTER}

    # Start at first splitter
    start = min(splitters)

    # DFS: returns number of paths from (row, col) to any exit
    path_count_per_cell = {}

    def dfs(row: int, col: int) -> int:
        """Internal DFS function."""
        if row >= field_height:
            # Exited bottom: one complete path
            return 1

        if (row, col) in path_count_per_cell:
            # If already in cache, no need to traverse again
            return path_count_per_cell[(row, col)]

        # Initialize path count
        path_count = 0

        if (row, col) in splitters:
            # Branch left and right
            if col > 0:
                path_count += dfs(row + 1, col - 1)
            if col + 1 < field_width:
                path_count += dfs(row + 1, col + 1)
        else:
            # Continue straight down
            path_count = dfs(row + 1, col)

        # Cache result
        path_count_per_cell[(row, col)] = path_count

        return path_count

    return dfs(*start)

In [13]:
# Correctness check
str(count_tachyon_paths(example.input_data)) == example.answer_b

True

In [14]:
# Performance check
iterative_time_b = time_solution(count_tachyon_paths, input_data)
print(f"The iterative implementation takes {iterative_time_b:.2f} ms per run.")

The iterative implementation takes 1.85 ms per run.


In [15]:
# Submit answer
puzzle.answer_b = count_tachyon_paths(input_data)

That's the right answer!  You are one gold star closer to decorating the North Pole.You have completed Day 7! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
